# Convolutional Layer with a Twist
> Modification of Conv2d that has a "built-in" capacity of learning to rotate/dilate the image. Easy plug-in for any CNN model.

- toc: true
- comments: true
- categories: [math, code, vision]

## Introducing...

I'm excited to introduce `conv_twist`, a replacement of—and, I argue, an improvement on—the good old **convolutional layer** widely used in Deep Learning models (rightfully referred to as ConvNets or CNNs) in Computer Vision. Famously introduced by Yann LeCun some 30 years ago into image classification, it became the source of the Deep Learning/Artificial Intelligence revolution with AlexNet in 2012. Rapid improvements on the architecture followed, most notably the ResNet of 2015. Recently attention has somewhat shifted away from image classification, but convolutional layers are still the bread and butter of any Computer Vision models. What more can be said about convolutional layers, one might ask? The answer is a little bit of mathematics.

For what it's worth, this is not peer-reviewed or published in any conference. If you'd like to give it a "review" on Twitter, feel free to do so.

## PyTorch implementation

Without further ado, here is one (crude) implementation of `conv_twist` in PyTorch, and you can easily swap out the 3x3 `Conv2d` in your model and plug this in, and train *from scratch* to see if it gives any improvement. Early success has been shown on [Imagenette/Imagewoof](https://github.com/fastai/imagenette) benchmark. (Help with testing on other models/datasets are greatly appreciated.)

In [1]:
import torch
import torch.nn as nn
import torchvision

class conv_twist(nn.Module):  # replacing 3x3 Conv2d
    def __init__(self, ni, nf, init_max=1.5, stride=1):
        super(conv_twist, self).__init__()
        self.conv = nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv_x = nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv_y = nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv_x.weight.data = (self.conv_x.weight - self.conv_x.weight.flip(2).flip(3)) / 2  # make conv_x a "first-order operator" by symmetrizing it
        self.conv_y.weight.data = self.conv_x.weight.transpose(2,3).flip(3)                      # make conv_y a 90 degree rotation of conv_x
        self.center_x = nn.Parameter(torch.Tensor(nf), requires_grad=True)
        self.center_y = nn.Parameter(torch.Tensor(nf), requires_grad=True)
        self.center_x.data.uniform_(-init_max, init_max)
        self.center_y.data.uniform_(-init_max, init_max)

    def forward(self, inpt):
        self.conv_x.weight.data = (self.conv_x.weight - self.conv_x.weight.flip(2).flip(3)) / 2  
        self.conv_y.weight.data = (self.conv_y.weight - self.conv_y.weight.flip(2).flip(3)) / 2
        out = self.conv(inpt)
        _, c, h, w = out.size()
        XX = torch.from_numpy(np.indices((1,h,w))[2]*2/w).type(out.dtype).to(out.device) - self.center_x.view(-1,1,1)
        YY = torch.from_numpy(np.indices((1,h,w))[1]*2/h).type(out.dtype).to(out.device) - self.center_y.view(-1,1,1)
        return out + (XX * self.conv_x(inpt) + YY * self.conv_y(inpt))

Let's take a look at the (initial) weights in such a `conv_twist` model:

In [2]:
model = conv_twist(1,3)
for name, para in model.named_parameters():
    print(name, para)

center_x Parameter containing:
tensor([ 0.6307, -1.4049, -0.1047], requires_grad=True)
center_y Parameter containing:
tensor([-0.1879,  0.6662,  1.1414], requires_grad=True)
conv.weight Parameter containing:
tensor([[[[-0.2893, -0.0233, -0.2600],
          [ 0.2398, -0.1335,  0.0162],
          [ 0.1423, -0.0215,  0.1642]]],


        [[[-0.1147, -0.0316, -0.0759],
          [ 0.2738, -0.2315,  0.1012],
          [ 0.0793,  0.3118,  0.0178]]],


        [[[ 0.1192,  0.2551,  0.3169],
          [-0.0515,  0.1144, -0.0921],
          [-0.1128, -0.1987, -0.0691]]]], requires_grad=True)
conv_x.weight Parameter containing:
tensor([[[[-0.0199, -0.2608, -0.1219],
          [-0.0325,  0.0000,  0.0325],
          [ 0.1219,  0.2608,  0.0199]]],


        [[[ 0.2378,  0.0571, -0.1488],
          [-0.1910,  0.0000,  0.1910],
          [ 0.1488, -0.0571, -0.2378]]],


        [[[-0.1464, -0.0639,  0.0475],
          [ 0.1355,  0.0000, -0.1355],
          [-0.0475,  0.0639,  0.1464]]]], requires_gra

If you take a look at the `conv_x` and `conv_y` weights, you'd notice that each 3x3 kernel is symmetric (the numbers on the opposite ends of the square are identical but with opposite signs, with the middle one always 0). That's the effect of "symmetrizing", done at each forward pass. You can also check that `conv_y` weights are initialized to be a 90 degree rotation of `conv_x`.

Why do I choose to initialize the weights this way, and what are `XX` and `YY`? Well, I'll try to explain everything later. For now it's important to note that `conv_twist` is a lot bigger than the standard `Conv2d` layer, but not as much as it appears to be. This particular implementation, if I had done it properly, has about twice as many trainable weights as a classic `Conv2d` layer.

## What do convolutions actually do?

The classic storyline in Neural Networks is that the convolution operator captures the spatial relation of the pixels (local features), so is particularly suited for image-related learning task; and secondly it has much fewer weights than a generic linear map (fully connected layer). Over the years, people learned that it's better to do away with kernels that are larger than 3x3 (and any biases), and to go deeper (i.e., many layers) instead, hence *deep* learning.

What is perhaps not well-known is that different 3x3 kernels have rather intuitive meanings, in terms of what it does to the image *overall*. For example, the Gaussian kernel in image processing "blurs" the image. We can do a little experiment to see:

In [3]:
G = torch.Tensor([[1,2,1],[2,4,2],[1,2,1]]) / 16
conv = nn.Conv2d(1,1,kernel_size=3, bias=False)
conv.weight.data = G
for name, param in conv.named_parameters():
    print(name, param)
    
# take a grayscale image, and feed it into the conv model. Display the result as an image side-by-side the original.

weight Parameter containing:
tensor([[0.0625, 0.1250, 0.0625],
        [0.1250, 0.2500, 0.1250],
        [0.0625, 0.1250, 0.0625]], requires_grad=True)


To illustrate the effect of other 3x3 kernels, it is best to choose the kernel close to the "identity", and to apply it many times. Try the following:

In [4]:
A = torch.Tensor([[-1,0,1],[-2,0,2],[-1,0,1]])
B = torch.Tensor([[1,2,1],[0,0,0],[-1,-2,-1]])
# These are the Sobel operator for (traditional) edge detection
I = torch.Tensor([[0,0,0],[0,1,0],[0,0,0]])
K = I + 0.01 * A
conv.weight.data = K
for name, param in conv.named_parameters():
    print(name, param)
    
# take a grayscale image, and feed it into the conv model. Display the result as an image side-by-side the original.

weight Parameter containing:
tensor([[-0.0100,  0.0000,  0.0100],
        [-0.0200,  1.0000,  0.0200],
        [-0.0100,  0.0000,  0.0100]], requires_grad=True)


Do you see the image gets shifted in the *x*- or *y*-direction? What does it all mean? Without defining terms, one would like to say that 
> the operators `A` and `B` *generate* translations in the *x*- and *y*-directions.

That means, once a neural network has learned to do a certain task, we can look at each of its 3x3 kernels and say it is doing a combination of (tiny bits of) blurring, translations, and possibly other transformations. That's what convolution *does* to the whole image. By the way, this "wholistic" point of view has the advantage that one can now "picture" the entire CNN with the same standard picture of a neural net of nodes and edges: each node is now a whole image (or feature map), and each edge is a convolution operator. (to insert image.)

What is missing from such a convolutional operator is **rotation** and **scaling**, as it is well known that CNNs are not rotation/scale-invariant, inasmuch as it is translation-invariant. There have been attempts at making a neural network invariant or *equivariant* for rotation and scaling, but to my limited understanding they are rather more like "forced" fixes than modifying the convolutional layer itself. For people with the right kind of mathematical backgrounds, the solution may come immediately to mind: the **(infinitesimal) generators** for rotation and scaling (**Lie algebra** of a **Lie group**), or **flow** of a **vector field**, or solving a first-order linear partial differential equation by the **method of characteristics**.

Let me first show how the `conv_twist` works before we get back to explain the mathematics, as simply as I could (not assuming any of the bold words above). Only then will we explain the details of the code.

## `conv_twist` at work (no training)

In addition to a normal Conv2d layer, the `conv_twist` is feeding the input to two other 3x3 Conv2d layer. To see the effect, let's turn off the `conv` weights

In [5]:
model = conv_twist(1,1)
model.conv.weight.data[:] = 0
model.center_x.data[:] = 0
model.center_y.data[:] = 0
model.conv_x.weight.data = I + 0.01 * A
model.conv_y.weight.data = I + 0.01 * B
for name, param in model.named_parameters():
    print(name, param)

center_x Parameter containing:
tensor([0.], requires_grad=True)
center_y Parameter containing:
tensor([0.], requires_grad=True)
conv.weight Parameter containing:
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], requires_grad=True)
conv_x.weight Parameter containing:
tensor([[-0.0100,  0.0000,  0.0100],
        [-0.0200,  1.0000,  0.0200],
        [-0.0100,  0.0000,  0.0100]], requires_grad=True)
conv_y.weight Parameter containing:
tensor([[ 0.0100,  0.0200,  0.0100],
        [ 0.0000,  1.0000,  0.0000],
        [-0.0100, -0.0200, -0.0100]], requires_grad=True)


## The mathematics

There are various ways that one could discuss this pheonomenon. But the first step is invariably to think of the image *not* as a collection of **discrete** data (pixels) but as a **continuous** object (function of two variables), and the pixels are "samples" of the function that give a good approximation for computations that we want to perform on the function.

For functions of continuous variables, the first thing you might want to do is taking (partial) derivatives in the x- or y-direction. How would you approximate that if you don't have full information of the function, but only the samples at discrete points?

In [6]:
import numpy as np
np.indices((1,4,5))[1]

array([[[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]]])